# NO2 emission during the COVID-19 pandemic

## About the data

A description of the data can be found in the docs/ folder in the repository.


## Research question
* Did the lockdown effect the NO2 emission in the Netherlands? 

## Table of contents

* ### <span style="color:#336699"><a href="#/2">1. Load the data</a></span>
    * <span style="color:#336699"><a href="#/3">1.1 Year 2020</a></span>
    * <span style="color:#336699"><a href="#/4">1.2 Year 2021</a></span>
    * <span style="color:#336699"><a href="#/5">1.3 Filter out stations with no data</a></span>
    * <span style="color:#336699"><a href="#/6">1.4 Impute missing data</a></span>
    * <span style="color:#336699"><a href="#/7">1.5 Meta data</a></span>
    * <span style="color:#336699"><a href="#/8">1.6 Covid data</a></span>
    * <span style="color:#336699"><a href="#/9">1.7 Tidy data format</a></span><br />

* ### <span style="color:#336699"><a href="#/10">2. Data inspection</a></span>
    * <span style="color:#336699"><a href="#/11">2.1 Data description</a></span>
    * <span style="color:#336699"><a href="#/12">2.2 Correlation Heatmap</a></span>
    * <span style="color:#336699"><a href="#/13">2.3 Distribution NO2 over the lockdown states - density plot</a></span>
    * <span style="color:#336699"><a href="#/14">2.4 Scatter plot</a></span>
    * <span style="color:#336699"><a href="#/15">2.5 Distribution NO2 over the lockdown states - violin plot</a></span>

* ### <span style="color:#336699"><a href="#/16">3. Check assumptions</a></span>
    * <span style="color:#336699"><a href="#/17">3.1 Check for normality - QQ plot</a></span>
    * <span style="color:#336699"><a href="#/18">3.2 Check for homogeneity - Levene's Test of Equality</a></span>
    * <span style="color:#336699"><a href="#/19">3.3 Summary</a></span>
    
* ### <span style="color:#336699"><a href="#/20">4 Test for differences</a></span>

* ### <span style="color:#336699"><a href="#/21">5 Conclusion</a></span>

* ### <span style="color:#336699"><a href="#/22">6 Code for the dashboard</a></span>
    * <span style="color:#336699"><a href="#/23">6.1 Geojson data</a></span>
    * <span style="color:#336699"><a href="#/24">6.2 NO2 emission map</a></span>
    * <span style="color:#336699"><a href="#/25">6.3 Interactive NO2 emission map</a></span>
    * <span style="color:#336699"><a href="#/26">6.4 Interactive NO2 emission plot</a></span>
    * <span style="color:#336699"><a href="#/27">6.5 Dashboard</a></span>

In [79]:
# IMPORTS
import sys
import numpy as np
import pandas as pd
import yaml
import json
from pathlib import Path
import glob
import os
from functools import partial
import random
import datetime as dt
from datetime import date

from IPython.display import Markdown as md

# geomap
import folium as fm
import folium.plugins
from folium import plugins
from folium.plugins import TimestampedGeoJson

from calendar import monthrange

# legend folium map
from branca.element import Template, MacroElement

# Data imputation
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge

In [80]:
import hvplot.pandas
import holoviews as hv
from holoviews import dim
from bokeh.io import output_notebook, output_file
from bokeh.io import export_png
from bokeh.plotting import figure, show
from bokeh.plotting import output_file, save
from bokeh.layouts import gridplot, column, layout, row
from bokeh.plotting import ColumnDataSource
from bokeh.models import DatetimeTickFormatter, DataTable, DateFormatter, TableColumn
from bokeh.models import CustomJS, Dropdown, Panel, Tabs
from bokeh.models import Span
from bokeh.transform import jitter
import param

import panel as pn
from panel.interact import fixed
import regex as re


pn.extension()
output_notebook()

Loading BokehJS ...

In [81]:
def get_config(file) -> dict:
    """
    Read in config file and return it as a dictionary.

    :parameter
    ----------
    file - String
        Location of config file
    
    :returns
    --------
    config - dict
        Configuration file in dictionary form.
    """
    try:
        with open(file, 'r') as stream:
            config = yaml.safe_load(stream)
    
        return config
    except FileNotFoundError as e:
        print(f"File: could not be found. Error {e}")
        sys.exit(1)

In [82]:
config = get_config("config.yaml")
data_dir = config['data']

# Utilities
Make these into functions not classes.

In [83]:
class Data:
    
    
    def read_CSV(self, file, *, skiprows=7, sep=";", encoding=None)-> pd.DataFrame:
        """
        Read a csv file in a pandas data frame.
        
        :parameters
        -----------
        file - Path
            Path to a file
        skiprows - int
            Number of rows to skip when reading in the data: optional, default = 7
        sep - str
            CSV seperator: optional, default=';'
        encoding - str
            Type of encoding: optional, default=None
            
        :returns
        --------
        df - pd.DataFrame
            A pandas data frame containing the CSV file
        """
        df = pd.read_csv(file, skiprows=skiprows, sep=sep, encoding=encoding)
        return df
    
    def rename_columns(self, df: pd.DataFrame, columns: dict) -> pd.DataFrame:
        """
        Rename columns of a data frame.
        
        :parameters
        -----------
        df - pd.DataFrame
            A pandas data frame
        columns - dict
            Dictionary conataining the names of the columns to rename and their new names
        """
        df.rename(columns = columns, inplace = True)
        return df
    
    
    def reformat_date_column(self, column:str, *, format_date: str = '%Y%m%d%H%M') -> pd.Series:
        """
        Reformat a date column. Remove white spaces inbetween the dates and typecast it to
        datetime type with the specified format.
        
        :parameters
        -----------
        column - str
            Name of column containg date information
        format_date - str
            Format of the date (i.e. %Y%m%d for year, month, day): optional, default='%Y%m%d%H%M'
            
        :returns
        --------
        column - pd.Series
            Reformated date column
        """
        try:
            column = column.str.replace(" ", "").str.replace(":","")
            column = pd.to_datetime(column.astype(str), format=format_date)
            return column
        except AttributeError as e:
            print(f"Date column: {column.name}, has already been reformatted. Error: {e}")
            return column

    
    def missing_value_filter(self, df: pd.DataFrame, p_missing: float) -> pd.DataFrame:
        """
        Keep only the columns that pass the maximum required % of missing values.

        :parameters
        -----------
        df - pd.DataFrame
            Data frame
        p_missing - float
            Maximum percentage missing values allowed for a column.

        :returns
        --------
        df - pd.DataFrame
            Filtered data frame based on % missing values.
        """
        df = df[df.columns[df.isnull().mean() < p_missing]]
        return df
    

    def tidy_df(self, df: pd.DataFrame, id_vars:list, value_vars:list, var_name:str, value_name:str) -> pd.DataFrame:
        """
        Create a tidy dataframe by transforming it from wide to long format.
        
        :parameters
        -----------
        df - pd.DataFrame
            Pandas data frame to transform
        id_vars - list
            Columns to keep
        value_vars - list
            Name of the columns to turn from wide to long
        var_name - str
            Name of the value vars column
        value_name - str
            Name of the new column containing the data of the value_vars columns
            
        :returns
        --------
        tidy_df - pd.DataFrame
            Pandas dataframe in long format
        """
        tidy_df = df.melt(id_vars=id_vars, value_vars=value_vars,
                          var_name = var_name,value_name = value_name)
        return tidy_df

    
data = Data()

In [84]:
class MetaData:
    
    def __init__(self, file):
        self.file = file
        self.df = self.read_data(self.file)
        self.df = self.remove_data(self.df, "StationsCode", axis = 0)
        self.create_lat_long_col()

    def read_data(self, file, *, nrows=6, sep=";", encoding='unicode_escape') -> pd.DataFrame:
        """
        Read in a the first 4 lines of a CSV file and then transposing it. This wil turn the 
        information above a CSV file into a meta data data frame.
                
        :parameters
        -----------
        file - Path
            Path to CSV file
        nrows - int
            Number of rows to read: optional, deault=6
        sep - str
            CSV seperator: optional, default=';'
        encoding - str
            Enconding type: optional, default='unicode_escape' 
            
        :returns
        --------
        df - pd.DataFrame
            A pandas data frame
        """
        df = pd.read_csv(file, nrows=nrows, sep=sep, encoding=encoding).iloc[:, 4:].T
        df = self.rename_columns(df)
        return df
        
    def rename_columns(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Use the first row to rename the columns.
        
        :parameters
        -----------
        df - pd.DataFrame
            A pandas data frame
            
        :returns
        --------
        df - pd.DataFrame
            A pandas data frame with renamed columns
        """
        df.columns = df.iloc[0,:]
        return df
    
    def create_lat_long_col(self) -> None:
        """
        Seperate the Latitude longitude column to create two seperate columns for both Latitude
        and Longitude.       
        """
        # Seperate the latitude and longitude column and assign it to their own column
        latitude_longitude = self.df["Latitude,Longitude"].str.strip("()").str.split(",", n = 1, expand = True)
        self.remove_data(self.df, "Latitude,Longitude", axis = 1) # Remove old column
        
        # Insert the chunk number column into the dataframe
        self.df.insert(1, column = "Latitude", value = latitude_longitude.iloc[:, 0])

        # Insert the patient id column into the dataframe
        self.df.insert(2, column = "Longitude", value = latitude_longitude.iloc[:, 1])
    
    @staticmethod
    def remove_data(df, name, *, axis = 1) -> pd.DataFrame:
        """
        Remove data from a pandas data frame.
        
        :parameters
        -----------
        df - pd.DataFrame
            A pandas data frame
        name - str
            Name of the column/row to remove
        axis - int
            1 = column, 0 = row
            
        :returns    
        --------
        df - pd.DataFrame
            A pandas data frame with a column/row removed
        """
        df.drop(name, axis = axis, inplace = True)
        return df
        
# metadata_instance = MetaData(file_2020)

# <a id="/2">1. Load data</a> 

First, load in the data from the year 2020. Then, load in the data of the year 2021 by combine all the seperate files into one data frame. 

## <a id="/3">1.1 Year 2020</a> 

In [85]:
file_2020 = Path(data_dir) / "2020" / "2020_NO2.csv"
# df_2020 = pd.read_csv(file_2020, skiprows=7, sep=";")
df_2020 = data.read_CSV(file_2020, skiprows=7, sep=";")
df_2020.head()

,Component,Bep.periode,Eenheid,Begindatumtijd,Einddatumtijd,NL01485,NL01487,NL01488,NL01489,NL01491,...,NL49022,NL49546,NL49551,NL49553,NL49561,NL49564,NL49565,NL49701,NL49703,NL49704
0,NO2,uur,�g/m�,20200101 00:00,20200101 01:00,45.8,48.5,47.8,36.9,41.4,...,30.4,26.8,35.2,21.5,27.8,13.1,22.9,49.2,21.6,17.1
1,NO2,uur,�g/m�,20200101 01:00,20200101 02:00,32.3,55.8,45.1,43.4,47.4,...,24.6,33.7,16.4,16.0,24.3,24.0,30.2,54.3,21.2,26.6
2,NO2,uur,�g/m�,20200101 02:00,20200101 03:00,32.3,42.8,32.9,39.3,37.4,...,22.9,39.6,23.9,24.8,27.6,26.9,30.6,50.9,22.4,32.6
3,NO2,uur,�g/m�,20200101 03:00,20200101 04:00,25.4,40.3,32.1,26.4,37.1,...,20.4,31.1,22.9,22.7,29.5,28.5,27.9,38.8,22.3,28.4
4,NO2,uur,�g/m�,20200101 04:00,20200101 05:00,24.3,31.3,24.3,23.1,27.1,...,25.1,26.7,26.3,25.0,29.1,25.8,27.0,29.1,25.8,28.7


### Clean data

In [86]:
# Rename the date columns
df_2020 = data.rename_columns(df_2020, columns = {"Begindatumtijd": "date_start", "Einddatumtijd": "date_end"})

# Set the datatype of the date and time colum to datetime.
df_2020['date_start'] = data.reformat_date_column(df_2020['date_start'], format_date='%Y%m%d%H%M')
df_2020['date_end'] = data.reformat_date_column(df_2020['date_end'], format_date='%Y%m%d%H%M')


df_2020.head()

,Component,Bep.periode,Eenheid,date_start,date_end,NL01485,NL01487,NL01488,NL01489,NL01491,...,NL49022,NL49546,NL49551,NL49553,NL49561,NL49564,NL49565,NL49701,NL49703,NL49704
0,NO2,uur,�g/m�,2020-01-01 00:00:00,2020-01-01 01:00:00,45.8,48.5,47.8,36.9,41.4,...,30.4,26.8,35.2,21.5,27.8,13.1,22.9,49.2,21.6,17.1
1,NO2,uur,�g/m�,2020-01-01 01:00:00,2020-01-01 02:00:00,32.3,55.8,45.1,43.4,47.4,...,24.6,33.7,16.4,16.0,24.3,24.0,30.2,54.3,21.2,26.6
2,NO2,uur,�g/m�,2020-01-01 02:00:00,2020-01-01 03:00:00,32.3,42.8,32.9,39.3,37.4,...,22.9,39.6,23.9,24.8,27.6,26.9,30.6,50.9,22.4,32.6
3,NO2,uur,�g/m�,2020-01-01 03:00:00,2020-01-01 04:00:00,25.4,40.3,32.1,26.4,37.1,...,20.4,31.1,22.9,22.7,29.5,28.5,27.9,38.8,22.3,28.4
4,NO2,uur,�g/m�,2020-01-01 04:00:00,2020-01-01 05:00:00,24.3,31.3,24.3,23.1,27.1,...,25.1,26.7,26.3,25.0,29.1,25.8,27.0,29.1,25.8,28.7


### Missing values

In [87]:
print(f"Missing values: {df_2020.iloc[:,5:].isnull().any()}")

Missing values: NL01485    True
NL01487    True
NL01488    True
NL01489    True
NL01491    True
           ... 
NL49564    True
NL49565    True
NL49701    True
NL49703    True
NL49704    True
Length: 73, dtype: bool


In [88]:
print(f" Check how many values there are missing per station:\n{df_2020.iloc[:,5:].isnull().sum()}")

 Check how many values there are missing per station:
NL01485    135
NL01487     30
NL01488     32
NL01489     29
NL01491    128
          ... 
NL49564     82
NL49565     68
NL49701     57
NL49703    130
NL49704     71
Length: 73, dtype: int64


In [89]:
p = 0.3
# Filter out all columns with to many missing values
df_2020 = data.missing_value_filter(df_2020, p)

df_2020.head(10)

,Component,Bep.periode,Eenheid,date_start,date_end,NL01485,NL01487,NL01488,NL01489,NL01491,...,NL49022,NL49546,NL49551,NL49553,NL49561,NL49564,NL49565,NL49701,NL49703,NL49704
0,NO2,uur,�g/m�,2020-01-01 00:00:00,2020-01-01 01:00:00,45.8,48.5,47.8,36.9,41.4,...,30.4,26.8,35.2,21.5,27.8,13.1,22.9,49.2,21.6,17.1
1,NO2,uur,�g/m�,2020-01-01 01:00:00,2020-01-01 02:00:00,32.3,55.8,45.1,43.4,47.4,...,24.6,33.7,16.4,16.0,24.3,24.0,30.2,54.3,21.2,26.6
2,NO2,uur,�g/m�,2020-01-01 02:00:00,2020-01-01 03:00:00,32.3,42.8,32.9,39.3,37.4,...,22.9,39.6,23.9,24.8,27.6,26.9,30.6,50.9,22.4,32.6
3,NO2,uur,�g/m�,2020-01-01 03:00:00,2020-01-01 04:00:00,25.4,40.3,32.1,26.4,37.1,...,20.4,31.1,22.9,22.7,29.5,28.5,27.9,38.8,22.3,28.4
4,NO2,uur,�g/m�,2020-01-01 04:00:00,2020-01-01 05:00:00,24.3,31.3,24.3,23.1,27.1,...,25.1,26.7,26.3,25.0,29.1,25.8,27.0,29.1,25.8,28.7
5,NO2,uur,�g/m�,2020-01-01 05:00:00,2020-01-01 06:00:00,22.5,35.9,30.4,30.9,29.8,...,23.8,31.8,27.8,29.5,24.3,23.2,22.9,31.0,25.7,28.3
6,NO2,uur,�g/m�,2020-01-01 06:00:00,2020-01-01 07:00:00,22.5,32.5,28.7,28.8,29.4,...,26.4,38.8,24.0,26.7,28.3,26.3,27.8,35.1,22.5,30.5
7,NO2,uur,�g/m�,2020-01-01 07:00:00,2020-01-01 08:00:00,25.2,23.7,18.9,22.9,29.6,...,25.3,35.3,25.8,26.9,28.0,32.3,34.4,30.4,27.3,31.2
8,NO2,uur,�g/m�,2020-01-01 08:00:00,2020-01-01 09:00:00,22.2,21.8,18.7,19.1,22.9,...,25.6,32.9,27.7,29.5,31.6,34.9,35.4,29.6,24.8,27.1
9,NO2,uur,�g/m�,2020-01-01 09:00:00,2020-01-01 10:00:00,22.9,20.8,17.0,22.2,19.5,...,27.4,29.3,31.8,34.5,32.3,27.7,27.5,30.3,31.2,28.6


## <a id="/4">1.2 Year 2021</a>  

In [90]:
# Map does not take keyword arugments: solution create a partial.
map_func = partial(data.read_CSV, skiprows=7, sep=";", encoding='unicode_escape')

# Merging the files 2021 csv files
list_2021 = glob.glob(os.path.join(data_dir, "2021", "*.csv"))
df_2021 = pd.concat(map(map_func, list_2021), ignore_index=True)
df_2021

,Component,Bep.periode,Eenheid,Begindatumtijd,Einddatumtijd,NL01485,NL01487,NL01488,NL01489,NL01491,...,NL49022,NL49546,NL49551,NL49553,NL49561,NL49564,NL49565,NL49701,NL49703,NL49704
0,NO2,uur,µg/m³,20210101 00:00,20210101 01:00,41.4,57.3,50.2,46.4,58.1,...,39.4,54.6,30.3,19.6,47.5,35.0,43.4,45.8,42.0,51.5
1,NO2,uur,µg/m³,20210101 01:00,20210101 02:00,41.8,51.4,47.8,43.0,56.2,...,36.0,53.4,39.7,17.9,49.2,33.6,44.4,40.2,43.3,48.9
2,NO2,uur,µg/m³,20210101 02:00,20210101 03:00,35.5,51.0,45.1,40.7,52.0,...,42.2,54.1,8.9,3.9,53.0,35.6,44.9,47.6,41.9,50.5
3,NO2,uur,µg/m³,20210101 03:00,20210101 04:00,20.4,52.9,49.9,45.3,54.6,...,41.2,55.1,9.0,3.0,56.6,38.9,56.0,42.7,39.3,47.1
4,NO2,uur,µg/m³,20210101 04:00,20210101 05:00,14.5,46.2,45.6,43.5,54.4,...,48.8,56.4,12.5,2.4,58.3,40.3,55.8,46.1,41.8,47.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8011,NO2,uur,µg/m³,20211130 19:00,20211130 20:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8012,NO2,uur,µg/m³,20211130 20:00,20211130 21:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8013,NO2,uur,µg/m³,20211130 21:00,20211130 22:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8014,NO2,uur,µg/m³,20211130 22:00,20211130 23:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Clean data

In [91]:
df_2021 = data.rename_columns(df_2021, columns = {"Begindatumtijd": "date_start", "Einddatumtijd": "date_end"})

# Set the datatype of the date and time colum to datetime.
df_2021['date_start'] = data.reformat_date_column(df_2021['date_start'], format_date='%Y%m%d%H%M')
df_2021['date_end'] = data.reformat_date_column(df_2021['date_end'], format_date='%Y%m%d%H%M')


df_2021.head()

,Component,Bep.periode,Eenheid,date_start,date_end,NL01485,NL01487,NL01488,NL01489,NL01491,...,NL49022,NL49546,NL49551,NL49553,NL49561,NL49564,NL49565,NL49701,NL49703,NL49704
0,NO2,uur,µg/m³,2021-01-01 00:00:00,2021-01-01 01:00:00,41.4,57.3,50.2,46.4,58.1,...,39.4,54.6,30.3,19.6,47.5,35.0,43.4,45.8,42.0,51.5
1,NO2,uur,µg/m³,2021-01-01 01:00:00,2021-01-01 02:00:00,41.8,51.4,47.8,43.0,56.2,...,36.0,53.4,39.7,17.9,49.2,33.6,44.4,40.2,43.3,48.9
2,NO2,uur,µg/m³,2021-01-01 02:00:00,2021-01-01 03:00:00,35.5,51.0,45.1,40.7,52.0,...,42.2,54.1,8.9,3.9,53.0,35.6,44.9,47.6,41.9,50.5
3,NO2,uur,µg/m³,2021-01-01 03:00:00,2021-01-01 04:00:00,20.4,52.9,49.9,45.3,54.6,...,41.2,55.1,9.0,3.0,56.6,38.9,56.0,42.7,39.3,47.1
4,NO2,uur,µg/m³,2021-01-01 04:00:00,2021-01-01 05:00:00,14.5,46.2,45.6,43.5,54.4,...,48.8,56.4,12.5,2.4,58.3,40.3,55.8,46.1,41.8,47.1


### Missing values

In [92]:
print(f"Missing values:\n{df_2020.iloc[:,5:].isnull().sum()}")

Missing values:
NL01485    135
NL01487     30
NL01488     32
NL01489     29
NL01491    128
          ... 
NL49564     82
NL49565     68
NL49701     57
NL49703    130
NL49704     71
Length: 73, dtype: int64


In [93]:
print(f"Check how many values there are missing per station:\n{df_2021.iloc[:,5:].isnull().sum()}")

Check how many values there are missing per station:
NL01485    3163
NL01487    2947
NL01488    3003
NL01489    2936
NL01491    3085
           ... 
NL49564    3787
NL49565    3709
NL49701    3687
NL49703    3899
NL49704    3836
Length: 73, dtype: int64


Inspecting the number of missing values per stations. It looks like that for the year 2021 a lot of stations contain a lot of missing values. Stations that contain to much missing values should be removed:

In [94]:
n_stations_2021 = len(df_2021.columns)

p = 0.15
# Filter out all columns with to many missing values
df_2021 = data.missing_value_filter(df_2021, p)

df_2021

,Component,Bep.periode,Eenheid,date_start,date_end,NL10107,NL10131,NL10133,NL10136,NL10138,...,NL10738,NL10741,NL10742,NL10807,NL10818,NL10918,NL10929,NL10934,NL10937,NL10938
0,NO2,uur,µg/m³,2021-01-01 00:00:00,2021-01-01 01:00:00,22.58,22.69,13.43,27.63,23.40,...,14.26,41.78,32.58,16.12,29.11,22.84,13.27,12.43,34.03,30.30
1,NO2,uur,µg/m³,2021-01-01 01:00:00,2021-01-01 02:00:00,21.79,23.09,16.36,26.93,19.52,...,12.45,42.19,33.53,15.62,32.72,20.13,14.64,15.12,24.56,22.76
2,NO2,uur,µg/m³,2021-01-01 02:00:00,2021-01-01 03:00:00,22.06,19.06,17.35,25.40,19.56,...,10.50,41.26,33.26,20.28,30.52,16.96,14.31,18.14,21.07,20.20
3,NO2,uur,µg/m³,2021-01-01 03:00:00,2021-01-01 04:00:00,20.73,25.61,15.83,25.33,20.44,...,9.52,40.00,32.02,24.03,30.22,18.91,17.05,13.87,20.22,18.98
4,NO2,uur,µg/m³,2021-01-01 04:00:00,2021-01-01 05:00:00,21.87,23.23,12.12,15.95,14.17,...,9.26,39.56,31.15,24.89,28.20,19.43,21.04,11.29,21.10,16.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8011,NO2,uur,µg/m³,2021-11-30 19:00:00,2021-11-30 20:00:00,12.36,13.51,9.59,10.96,8.26,...,17.87,24.45,28.44,4.90,7.32,4.97,3.47,4.35,29.99,11.30
8012,NO2,uur,µg/m³,2021-11-30 20:00:00,2021-11-30 21:00:00,11.26,15.39,7.32,9.31,7.86,...,15.57,23.51,28.75,6.93,6.52,3.33,5.33,8.37,28.89,12.02
8013,NO2,uur,µg/m³,2021-11-30 21:00:00,2021-11-30 22:00:00,9.34,13.00,6.84,12.06,10.17,...,10.47,18.70,24.28,8.56,6.61,15.93,6.43,7.35,17.58,10.20
8014,NO2,uur,µg/m³,2021-11-30 22:00:00,2021-11-30 23:00:00,7.48,9.65,8.42,8.17,8.97,...,9.80,17.22,24.54,13.63,11.27,14.04,6.10,5.42,15.49,7.29


In [95]:
# @hidden_cell
md(f"Number of stations removed: {n_stations_2021 - len(df_2021.columns)}")

Number of stations removed: 30

## <a id="/5">1.3 Filter out stations with no data</a>  

In [96]:
# Column names of the 2021 data frame
cols_to_keep = df_2021.columns.values.tolist()

# Only keep relevant columns
df_2020 = df_2020.loc[:,cols_to_keep]
df_2020.head()

,Component,Bep.periode,Eenheid,date_start,date_end,NL10107,NL10131,NL10133,NL10136,NL10138,...,NL10738,NL10741,NL10742,NL10807,NL10818,NL10918,NL10929,NL10934,NL10937,NL10938
0,NO2,uur,�g/m�,2020-01-01 00:00:00,2020-01-01 01:00:00,38.78,20.94,27.48,40.33,38.70,...,32.69,42.24,44.22,6.14,10.44,30.06,29.30,22.68,37.35,44.95
1,NO2,uur,�g/m�,2020-01-01 01:00:00,2020-01-01 02:00:00,38.31,26.56,37.05,43.46,37.70,...,21.93,37.59,30.06,9.11,6.53,21.39,25.27,25.78,35.03,35.87
2,NO2,uur,�g/m�,2020-01-01 02:00:00,2020-01-01 03:00:00,37.06,34.22,38.43,38.72,41.50,...,23.22,39.67,31.79,12.25,7.75,9.38,22.14,26.87,30.58,30.06
3,NO2,uur,�g/m�,2020-01-01 03:00:00,2020-01-01 04:00:00,35.16,34.82,37.74,33.25,34.47,...,22.20,38.14,32.77,15.16,8.96,6.79,16.38,24.02,22.66,22.74
4,NO2,uur,�g/m�,2020-01-01 04:00:00,2020-01-01 05:00:00,32.31,36.92,36.39,36.58,37.55,...,28.42,39.48,36.21,14.39,11.20,6.16,11.00,11.84,20.93,16.92


## <a id="/6">1.4 Impute missing data</a>  

The stations that passed the missing data filter still have some missing values. These values should be either interpolated or imputed. Data imputation was chosen.

Performance of four data imputation methods were measured on air quality measurement data (CO, O3, PM10, SO2, NOx, NO2). The four methods were, Mean Top Bottom, Neirest Neighbours, Linear regression, and Multiple imputation[1]. This study concluded that the Mean Top Bottom approach was best to impute air quality measurement data. However, the data that was used for that study had mostly small gaps in the data, for example one or three hours. Whereas the data for this project contains more larger gaps(i.e. 40 hours). While testing the Mean Top Bottom approach it became clear that it could not handle large gaps in the data. Therefore, another approach was tested; regression based data imputation. Measuring stations that had a relationship (correlation >= 0.7) with the station of interest were used to create a predictive model to fill the missing values. This method seems to work well, which is in contrast to the findings of N. A. Zakaria[1]. They concluded that the Linear regression method performed the worst out of the four methods that were tested. 

[1] https://www.semanticscholar.org/paper/Imputation-methods-for-filling-missing-data-in-air-Zakaria-Noor/068aef2863fa856e8498f74674ecb4806df88c93

In [97]:
def calculate_corr(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the correlation between different columns of a data frame.
    
    :parameters
    -----------
    df - pd.DataFrame
        Data frame
    
    :returns
    --------
    c - pd.DataFrame
        Correlation matrix
    """
    c = df.corr().abs()
    return c

## 2020

In [98]:
# Get the names of the different stations
stations = df_2020.columns[df_2020.columns.str.contains("NL")]

# calculate the correlation between each station for the year 2020
cor_stations_2020 = calculate_corr(df_2020.loc[:,stations])

cor_stations_2020.head(5)

,NL10107,NL10131,NL10133,NL10136,NL10138,NL10230,NL10235,NL10236,NL10237,NL10240,...,NL10738,NL10741,NL10742,NL10807,NL10818,NL10918,NL10929,NL10934,NL10937,NL10938
NL10107,1.000000,0.786493,0.715316,0.729730,0.814773,0.464179,0.537842,0.763899,0.715351,0.636080,...,0.667494,0.625002,0.740633,0.602159,0.586028,0.485667,0.526274,0.458137,0.467106,0.497633
NL10131,0.786493,1.000000,0.607234,0.663923,0.718681,0.520781,0.529592,0.764157,0.680960,0.635263,...,0.732080,0.627954,0.757683,0.649090,0.607254,0.505326,0.590060,0.471963,0.435828,0.495598
NL10133,0.715316,0.607234,1.000000,0.681717,0.793526,0.356400,0.484573,0.646003,0.581961,0.497754,...,0.567923,0.508650,0.611045,0.538856,0.515437,0.407384,0.429012,0.399553,0.457390,0.472347
NL10136,0.729730,0.663923,0.681717,1.000000,0.848987,0.382280,0.382627,0.763496,0.727545,0.675120,...,0.553697,0.727328,0.709327,0.460940,0.493645,0.366129,0.388438,0.370780,0.567287,0.412287
NL10138,0.814773,0.718681,0.793526,0.848987,1.000000,0.409894,0.496793,0.766782,0.707235,0.632761,...,0.644634,0.638456,0.732766,0.582537,0.589076,0.473212,0.492728,0.458794,0.505232,0.506966


In [99]:
# find columns to correlate well with a certain station
def find_correlated_stations(cor_df: pd.DataFrame, station: str, cor_threshold: float) -> list:
    """
    Find stations that are closely related (highly correlated) to a specific station.
    
    :parameters
    -----------
    cor_df - pd.DataFrame
        Correlation matrix of the stations
    station - str
        Station of interest
    cor_threshold - float
        Cut off to be considered highly correlated
        
    :returns
    --------
    stations - list
        List of correlated stations
    """
    corr_station = cor_df.loc[:,cor_df.columns == station]
    stations = corr_station[corr_station[station] > cor_threshold].index.values.tolist()
    return stations

stations_NL10138 = find_correlated_stations(cor_stations_2020, "NL10138", 0.7)

In [100]:
# Put this in a class
def data_impute_regression(col: pd.Series, df: pd.DataFrame) -> pd.Series:
    """
    Use regression-based model to fill in the missing value for a station. Features used for 
    predictions are the stations that are highly correlated to one another. 
    
    :parameters
    -----------
    col - pd.Series
        Column of a data frame
    df - pd.DataFrame
        Data frame containing the features
    
    :returns
    --------
    imputed_col - pd.Series
        Imputed column of interest
    """
    stations = find_correlated_stations(cor_stations_2020, col.name, 0.7)
    imputer = IterativeImputer(BayesianRidge())
    imputed_df = pd.DataFrame(imputer.fit_transform(df[stations]), columns = stations)
    imputed_col = imputed_df[col.name]
    return imputed_col

In [101]:
# Imput the data for columns with missing values
df_2020_imp = df_2020.loc[:,stations].apply(lambda x: data_impute_regression(col=x, df=df_2020), axis=0)

cols_to_keep = ["Component", "Eenheid", "date_start", "date_end"]
df_2020_imp = pd.concat([df_2020.loc[:,cols_to_keep], df_2020_imp], axis=1)

C:\Users\stijn\anaconda3\envs\DSLS_prog1\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [102]:
df_2020_imp.head(5)

,Component,Eenheid,date_start,date_end,NL10107,NL10131,NL10133,NL10136,NL10138,NL10230,...,NL10738,NL10741,NL10742,NL10807,NL10818,NL10918,NL10929,NL10934,NL10937,NL10938
0,NO2,�g/m�,2020-01-01 00:00:00,2020-01-01 01:00:00,38.78,20.94,27.48,40.33,38.70,23.65,...,32.69,42.24,44.22,6.14,10.44,30.06,29.30,22.68,37.35,44.95
1,NO2,�g/m�,2020-01-01 01:00:00,2020-01-01 02:00:00,38.31,26.56,37.05,43.46,37.70,35.95,...,21.93,37.59,30.06,9.11,6.53,21.39,25.27,25.78,35.03,35.87
2,NO2,�g/m�,2020-01-01 02:00:00,2020-01-01 03:00:00,37.06,34.22,38.43,38.72,41.50,28.47,...,23.22,39.67,31.79,12.25,7.75,9.38,22.14,26.87,30.58,30.06
3,NO2,�g/m�,2020-01-01 03:00:00,2020-01-01 04:00:00,35.16,34.82,37.74,33.25,34.47,22.57,...,22.20,38.14,32.77,15.16,8.96,6.79,16.38,24.02,22.66,22.74
4,NO2,�g/m�,2020-01-01 04:00:00,2020-01-01 05:00:00,32.31,36.92,36.39,36.58,37.55,18.47,...,28.42,39.48,36.21,14.39,11.20,6.16,11.00,11.84,20.93,16.92


## 2021

In [103]:
# calculate the correlation between each station for the year 2021
cor_stations_2021 = calculate_corr(df_2021.loc[:,stations])

In [104]:
# Imput the data for columns with missing values
df_2021_imp = df_2021.loc[:,stations].apply(lambda x: data_impute_regression(col=x, df=df_2021), axis=0)
cols_to_keep = ["Component", "Eenheid", "date_start", "date_end"]
df_2021_imp = pd.concat([df_2021.loc[:,cols_to_keep], df_2021_imp], axis=1)

C:\Users\stijn\anaconda3\envs\DSLS_prog1\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


## <a id="/7">1.5 Meta Data</a>  

In [105]:
meta_data = MetaData(file_2020)

df_meta_data = meta_data.df

# Change dtypes
df_meta_data.Latitude = pd.to_numeric(df_meta_data.Latitude)
df_meta_data.Longitude = pd.to_numeric(df_meta_data.Longitude)
df_meta_data.head(5)

StationsCode,Stationsnaam,Latitude,Longitude,Stationsgebied,Stationstype,Meetprincipe,Meetopstelling
NL01485,Hoogvliet-Leemkuil,51.867411,4.355242,stad,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL01487,Rotterdam Zuid-Pleinweg,51.891147,4.480690,regionaal,verkeer,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL01488,Rotterdam Zuid-Zwartewaalstraat,51.893617,4.487528,stad,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL01489,Ridderkerk-Hogeweg,51.869431,4.580058,stad,verkeer,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
NL01491,Rotterdam-Oost Sidelinge A13,51.938472,4.430692,stad,verkeer,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser


## <a id="/8">1.6 Covid data</a>  

Now that the data has been cleaned another column can be created that indicates whether or not a specific date was during a lockdown or not. To be able to do this the different dates of the lockdowns need to be known. These were derived from the offical website of the Dutch goverment[1] and wikipedia[2]. Here is a list of the different lockdowns in the Netherlands:

Lockdown:
* 14 oktober 2020 - 14 december 2020: partially lockdown
* 15 december 2020 - 27 april 2021: hard lockdown
* 28 april 2021 - 25 september 2021: a few restrictions
* 26 september 2021 - 26 november 2021: restrictions
* 27 november 2021 - 17 december 2021: lockdown in the evening
* 18 december 2021 -  5 januari 2022: lockdown

For the purpose of this study the differences of the lockdowns were not accounted for and were treated as equal. 

[1] https://www.rijksoverheid.nl/onderwerpen/coronavirus-tijdlijn  
[2] https://nl.wikipedia.org/wiki/Maatregelen_tijdens_de_coronacrisis_in_Nederland#Maatregelen_naar_datum_van_aankondiging

In [106]:
def get_lockdown_state(date: dt) -> bool:
    """
    Check for a date if it was during a lockdown.
    
    :parameters
    -----------
    date - datetime
        Date - format: YYYY-MM-DD
        
    :returns
    --------
    lockdown_one - bool
        Date is during first lockdown: True or False
    lockdown_two - bool
        Date is during second lockdown: True or False
    lockdown_three - bool
        Date is during third lockdown: True or False
    """
    date = pd.to_datetime(date, utc=True)
    
    lockdown_one = (pd.to_datetime('2020-10-14', utc= True) <= date) & (date <= pd.to_datetime('2020-12-14', utc= True))
    lockdown_two = (pd.to_datetime('2020-12-15', utc= True) <= date) & (date <= pd.to_datetime('2021-04-27', utc= True))
    lockdown_three = (pd.to_datetime('2021-12-18', utc= True) <= date) & (date <= pd.to_datetime('2022-01-05', utc= True))

    return lockdown_one, lockdown_two, lockdown_three


In [107]:
# l1, l2, l3 = check_lockdown(df_2020['date_start'].dt.date, df_2020['date_end'].dt.date)
l1, l2, l3 = get_lockdown_state(df_2020['date_start'].dt.date)

df_2020_imp["Lockdown"] = np.where(l1 | l2 | l3, "Lockdown", "No-lockdown")

# Change dtype
df_2020_imp["Lockdown"]  = df_2020_imp["Lockdown"].astype("category")

In [108]:
# l1, l2, l3 = check_lockdown(df_2021['date_start'].dt.date, df_2021['date_end'].dt.date)
l1, l2, l3 = get_lockdown_state(df_2021['date_start'].dt.date)

df_2021_imp.loc[:,"Lockdown"] = np.where(l1 | l2 | l3, "Lockdown", "No-lockdown")

# Change dtype
df_2021_imp.loc[:,"Lockdown"]  = df_2021_imp.loc[:,"Lockdown"].astype("category")

In [109]:
df_2021_imp.head(5)

,Component,Eenheid,date_start,date_end,NL10107,NL10131,NL10133,NL10136,NL10138,NL10230,...,NL10741,NL10742,NL10807,NL10818,NL10918,NL10929,NL10934,NL10937,NL10938,Lockdown
0,NO2,µg/m³,2021-01-01 00:00:00,2021-01-01 01:00:00,22.58,22.69,13.43,27.63,23.40,35.88,...,41.78,32.58,16.12,29.11,22.84,13.27,12.43,34.03,30.30,Lockdown
1,NO2,µg/m³,2021-01-01 01:00:00,2021-01-01 02:00:00,21.79,23.09,16.36,26.93,19.52,38.31,...,42.19,33.53,15.62,32.72,20.13,14.64,15.12,24.56,22.76,Lockdown
2,NO2,µg/m³,2021-01-01 02:00:00,2021-01-01 03:00:00,22.06,19.06,17.35,25.40,19.56,35.85,...,41.26,33.26,20.28,30.52,16.96,14.31,18.14,21.07,20.20,Lockdown
3,NO2,µg/m³,2021-01-01 03:00:00,2021-01-01 04:00:00,20.73,25.61,15.83,25.33,20.44,28.71,...,40.00,32.02,24.03,30.22,18.91,17.05,13.87,20.22,18.98,Lockdown
4,NO2,µg/m³,2021-01-01 04:00:00,2021-01-01 05:00:00,21.87,23.23,12.12,15.95,14.17,21.38,...,39.56,31.15,24.89,28.20,19.43,21.04,11.29,21.10,16.14,Lockdown


### Merge covid cases

In [110]:
covid_data = pd.read_csv(Path(data_dir + "time_series_covid19_confirmed_global.csv"), sep=",")
covid_NL = covid_data[(covid_data["Country/Region"]=="Netherlands") & (covid_data["Province/State"].isnull())]

In [111]:
# Melt the covid data frame
covid_NL = data.tidy_df(covid_NL, id_vars=["Country/Region", "Lat", "Long"], value_vars=covid_NL.columns[4:],
                       var_name = "Date", value_name = "covid_patients")

covid_NL.Date = pd.to_datetime(covid_NL.Date.astype(str))
# Drop unwanted columns
covid_NL.drop(list(covid_NL.columns[:3]), axis=1, inplace=True)

In [112]:
covid_NL.head(5)

,Date,covid_patients
0,2020-01-22,0
1,2020-01-23,0
2,2020-01-24,0
3,2020-01-25,0
4,2020-01-26,0


### 2020

In [113]:
# Merge df
NO2_cov_2020 = pd.merge_asof(df_2020_imp, covid_NL, left_on='date_start', right_on="Date")

NO2_cov_2020.drop("Date", axis=1, inplace=True)

NO2_cov_2020.covid_patients = NO2_cov_2020.covid_patients.fillna(0)

In [114]:
NO2_cov_2020.head(5)

,Component,Eenheid,date_start,date_end,NL10107,NL10131,NL10133,NL10136,NL10138,NL10230,...,NL10742,NL10807,NL10818,NL10918,NL10929,NL10934,NL10937,NL10938,Lockdown,covid_patients
0,NO2,�g/m�,2020-01-01 00:00:00,2020-01-01 01:00:00,38.78,20.94,27.48,40.33,38.70,23.65,...,44.22,6.14,10.44,30.06,29.30,22.68,37.35,44.95,No-lockdown,0.0
1,NO2,�g/m�,2020-01-01 01:00:00,2020-01-01 02:00:00,38.31,26.56,37.05,43.46,37.70,35.95,...,30.06,9.11,6.53,21.39,25.27,25.78,35.03,35.87,No-lockdown,0.0
2,NO2,�g/m�,2020-01-01 02:00:00,2020-01-01 03:00:00,37.06,34.22,38.43,38.72,41.50,28.47,...,31.79,12.25,7.75,9.38,22.14,26.87,30.58,30.06,No-lockdown,0.0
3,NO2,�g/m�,2020-01-01 03:00:00,2020-01-01 04:00:00,35.16,34.82,37.74,33.25,34.47,22.57,...,32.77,15.16,8.96,6.79,16.38,24.02,22.66,22.74,No-lockdown,0.0
4,NO2,�g/m�,2020-01-01 04:00:00,2020-01-01 05:00:00,32.31,36.92,36.39,36.58,37.55,18.47,...,36.21,14.39,11.20,6.16,11.00,11.84,20.93,16.92,No-lockdown,0.0


### 2021

In [115]:
# Merge df
NO2_cov_2021 = pd.merge_asof(df_2021_imp, covid_NL, left_on='date_start', right_on="Date")

NO2_cov_2021.drop("Date", axis=1, inplace=True)

NO2_cov_2021.covid_patients = NO2_cov_2021.covid_patients.fillna(0)

In [116]:
NO2_cov_2021.head(5)

,Component,Eenheid,date_start,date_end,NL10107,NL10131,NL10133,NL10136,NL10138,NL10230,...,NL10742,NL10807,NL10818,NL10918,NL10929,NL10934,NL10937,NL10938,Lockdown,covid_patients
0,NO2,µg/m³,2021-01-01 00:00:00,2021-01-01 01:00:00,22.58,22.69,13.43,27.63,23.40,35.88,...,32.58,16.12,29.11,22.84,13.27,12.43,34.03,30.30,Lockdown,805164
1,NO2,µg/m³,2021-01-01 01:00:00,2021-01-01 02:00:00,21.79,23.09,16.36,26.93,19.52,38.31,...,33.53,15.62,32.72,20.13,14.64,15.12,24.56,22.76,Lockdown,805164
2,NO2,µg/m³,2021-01-01 02:00:00,2021-01-01 03:00:00,22.06,19.06,17.35,25.40,19.56,35.85,...,33.26,20.28,30.52,16.96,14.31,18.14,21.07,20.20,Lockdown,805164
3,NO2,µg/m³,2021-01-01 03:00:00,2021-01-01 04:00:00,20.73,25.61,15.83,25.33,20.44,28.71,...,32.02,24.03,30.22,18.91,17.05,13.87,20.22,18.98,Lockdown,805164
4,NO2,µg/m³,2021-01-01 04:00:00,2021-01-01 05:00:00,21.87,23.23,12.12,15.95,14.17,21.38,...,31.15,24.89,28.20,19.43,21.04,11.29,21.10,16.14,Lockdown,805164


## <a id="/9">1.7 Tidy data format</a>  

To be able to work with this data set we need to transform it from wide to long format.

### 2020

In [117]:
id_vars=["date_start", "date_end", "Lockdown", "covid_patients"]
# Get all the stations
value_vars = NO2_cov_2020.columns[NO2_cov_2020.columns.str.contains("NL")]

# Create tidy data frame
tidy_2020 = data.tidy_df(NO2_cov_2020, id_vars=id_vars, value_vars=value_vars,
                       var_name = "site", value_name = "NO2")
tidy_2020

,date_start,date_end,Lockdown,covid_patients,site,NO2
0,2020-01-01 00:00:00,2020-01-01 01:00:00,No-lockdown,0.0,NL10107,38.78
1,2020-01-01 01:00:00,2020-01-01 02:00:00,No-lockdown,0.0,NL10107,38.31
2,2020-01-01 02:00:00,2020-01-01 03:00:00,No-lockdown,0.0,NL10107,37.06
3,2020-01-01 03:00:00,2020-01-01 04:00:00,No-lockdown,0.0,NL10107,35.16
4,2020-01-01 04:00:00,2020-01-01 05:00:00,No-lockdown,0.0,NL10107,32.31
...,...,...,...,...,...,...
377707,2020-12-31 19:00:00,2020-12-31 20:00:00,Lockdown,796981.0,NL10938,22.49
377708,2020-12-31 20:00:00,2020-12-31 21:00:00,Lockdown,796981.0,NL10938,32.30
377709,2020-12-31 21:00:00,2020-12-31 22:00:00,Lockdown,796981.0,NL10938,28.30
377710,2020-12-31 22:00:00,2020-12-31 23:00:00,Lockdown,796981.0,NL10938,30.09


### 2021

In [118]:
id_vars=["date_start", "date_end", "Lockdown", "covid_patients"]
# Get all the stations
value_vars = NO2_cov_2021.columns[NO2_cov_2021.columns.str.contains("NL")]

# Create tidy data frame
tidy_2021 = data.tidy_df(NO2_cov_2021, id_vars=id_vars, value_vars=value_vars,
                       var_name = "site", value_name = "NO2")
tidy_2021

,date_start,date_end,Lockdown,covid_patients,site,NO2
0,2021-01-01 00:00:00,2021-01-01 01:00:00,Lockdown,805164,NL10107,22.58
1,2021-01-01 01:00:00,2021-01-01 02:00:00,Lockdown,805164,NL10107,21.79
2,2021-01-01 02:00:00,2021-01-01 03:00:00,Lockdown,805164,NL10107,22.06
3,2021-01-01 03:00:00,2021-01-01 04:00:00,Lockdown,805164,NL10107,20.73
4,2021-01-01 04:00:00,2021-01-01 05:00:00,Lockdown,805164,NL10107,21.87
...,...,...,...,...,...,...
344683,2021-11-30 19:00:00,2021-11-30 20:00:00,No-lockdown,2643176,NL10938,11.30
344684,2021-11-30 20:00:00,2021-11-30 21:00:00,No-lockdown,2643176,NL10938,12.02
344685,2021-11-30 21:00:00,2021-11-30 22:00:00,No-lockdown,2643176,NL10938,10.20
344686,2021-11-30 22:00:00,2021-11-30 23:00:00,No-lockdown,2643176,NL10938,7.29


# <a id="/10">2 Data inspection</a>  

##  <a id="/11">2.1 Data description</a>  

In [119]:
tidy_2020.describe()

,covid_patients,NO2
count,377712.000000,377712.000000
mean,137850.530055,15.431501
std,200904.721613,12.268614
min,0.000000,-4.940000
25%,13614.000000,6.682349
50%,50373.500000,11.980000
75%,124097.000000,20.610000
max,796981.000000,315.040000


In [120]:
tidy_2021.describe()

,covid_patients,NO2
count,3.446880e+05,344688.000000
mean,1.622148e+06,14.740487
std,4.373339e+05,12.130840
min,8.051640e+05,-4.710000
25%,1.228647e+06,6.230000
50%,1.676176e+06,11.270000
75%,1.961585e+06,19.590000
max,2.643176e+06,355.130000


When comparing the basic statistics of the two data frames it is clear that the year 2021 contains overall more covid cases. However, the statistics about the NO2 emission looks to be somewhat the same for both years. Something notably is that the NO2 emission values has a negative value as its minimum value, which is something that is not expected. This is most probably caused by measuring inaccuracies. These measuring inaccuracies can be caused by, for example rapidly changing wheater conditions (i.e. humidity, temperature)[1].The max values for both years are far higher then the mean. This might indicate that there is an outlier present. However, according to the world health organiztaion(WHO) the maximum value of the hourly mean of NO2 emission might be much higher compared to the annual mean[2]. Indicating that this probably is not an outlier. In addition, a document of the WHO about Ambient (Outdoor) AirPollution Guidelines states that a hourly mean of 200 μg/m3 is normal[3]. These high values were also not introduced by data imputation. This was validated by checking the maximum values before the data imputation step. 


[1] https://www.luchtmeetnet.nl/informatie/overige/negatieve-waarden  
[2] https://www.ncbi.nlm.nih.gov/books/NBK138707/  
[3] https://environmentaldevices.com/wp-content/uploads/2020/05/WHO-Guidlines.pdf

## <a id="/12">2.2 Correlation heatmap</a>  

In [121]:
from bokeh.models import (BasicTicker, ColorBar, ColumnDataSource,
                          LinearColorMapper, PrintfTickFormatter,)
from bokeh.transform import transform
from bokeh.palettes import Viridis256

def plot_heatmap(df_cor, sample):
    """
    Create a heatmap plot.
    
    :parameters
    -----------
    df_cor - pd.DataFrame
        Correlation data frame
    sample - str
        Name of the sample
        
    :returns
    --------
    p - figure
        Heatmap 
    """
    #reshape
    dfc = pd.DataFrame(df_cor.stack(), columns=['r']).reset_index()

    y_range = (list(reversed(df_cor.columns)))
    x_range = (list(df_cor.index))

    source = ColumnDataSource(dfc)

    #create colormapper 
    mapper = LinearColorMapper(palette=Viridis256, low=dfc.r.min(), high=dfc.r.max())

    #create plot
    p = figure(title=f"Correlation heatmap {sample}", plot_width=500, plot_height=450,
               x_range=x_range, y_range=y_range, x_axis_location="above", toolbar_location=None)

    #use mapper to fill the rectangles in the plot
    p.rect(x="level_0", y="level_1", width=1, height=1, source=source,
           line_color=None, fill_color=transform('r', mapper))

    #create and add colorbar to the right
    color_bar = ColorBar(color_mapper=mapper, location=(0, 0),
                         ticker=BasicTicker(desired_num_ticks=len(x_range)), 
                         formatter=PrintfTickFormatter(format="%.1f"))
    p.add_layout(color_bar, 'right')

    #draw axis
    p.axis.axis_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.major_label_text_font_size = "10px"
    p.axis.major_label_standoff = 0
    p.xaxis.major_label_orientation = 1.55

    return p 

heatmap_2021 = plot_heatmap(cor_stations_2021, "2021")
show(heatmap_2021)

In [122]:
heatmap_2020 = plot_heatmap(cor_stations_2020, "2020")
show(heatmap_2020)

From the heatmap we can see that some stations are more corelated to each other then others. This is probably because some stations are located close to each other and others are in a complete different region. Similar NO2 emission measurements are expected for stations that are closely related, espically if they are located in the same city. 

## <a id="/13">2.3 Distribution NO2 over the lockdown states - density plot</a>  

In [123]:
autocomplete_station_1 = pn.widgets.AutocompleteInput(
    name='Station', options=stations.values.tolist() + ["All"],
    value=stations.values.tolist()[0],
    placeholder='ID of station..')

years_df_1 = {"2020":tidy_2020,
           "2021":tidy_2021}

select_year_1 = pn.widgets.Select(name="Year",
                                options=["2020", "2021"])

In [124]:

# @pn.depends(station=autocomplete_station_1, year=select_year_1)
def plot_density1(column, by, colors, year="2020", station="NL10107") -> figure:
    """
    Create a density plot for the specific site.
    
    :parameters
    -----------
    df - pd.DataFrame
        Data frame
    site - str
        Name of the site
    columns - list
        Which columns to use to display density
    by - str
        By which column to create density plot
    colors - list
        List of colors to used.
        
    :returns
    --------
    p - figure
        Density plot
    """
    df = years_df_1[year]
    
    if station == "All":
        p = df.loc[:, [column, by]].hvplot.kde(by=by, legend = 'top_right', color = colors)
        p.opts(title=f"Density plot for {station.lower()} stations - {year}", xlabel = "NO2 (µg/m³)")
    else:
        p = df.loc[df['site'] == station, [column,by]].hvplot.kde(by=by, legend = 'top_right', color = colors)
        p.opts(title=f"Density plot for station {station} - {year}", xlabel = "NO2 (µg/m³)")

    return p
    
# Fixed: https://panel.holoviz.org/user_guide/Interact.html
density_NO2 = pn.interact(plot_density1, column=fixed("NO2"), by=fixed("Lockdown"), 
                     colors=fixed(["red", "green"]), year = select_year_1, station=autocomplete_station_1)

pn.Column(pn.Row(density_NO2[0][1], density_NO2[0][0]),
         pn.Row(density_NO2[1]))

Column
    [0] Row
        [0] AutocompleteInput(name='Station', options=['NL10107', 'NL10131', ...], placeholder='ID of station..', value='NL10107')
        [1] Select(name='Year', options=['2020', '2021'], value='2020')
    [1] Row
        [0] Row
            [0] HoloViews(NdOverlay, name='interactive02782')

Using the two widgets we can select the staion and year of interest. From the density plot of station "NL10107" and the year 2020 we can see that there is a different distribution for both lockdown and no-lockdown. However, for the year 2021 there is not a clear difference between the two. 

## <a id="/14">2.4 Scatter plot</a>  

In [125]:
autocomplete_station = pn.widgets.AutocompleteInput(
    name='Station', options=stations.values.tolist(),
    value=stations.values.tolist()[0],
    placeholder='ID of station..')

In [126]:
years_df = {"2020":tidy_2020,
           "2021":tidy_2021}

select_year = pn.widgets.Select(name="Year",
                                options=["2020", "2021"])


@pn.depends(site=autocomplete_station, year=select_year)
def create_scatter(year, site="NL10247"):
    """
    Create a scatter plot to show the relation between how tired you feel and hours slept.
    
    :parameters
    -----------
    df - pandas.DataFrame
        Data frame
    
    :returns
    --------
    p - figure
        Scatter plot
    """
    df= years_df[year]

    # Create two seperate data frames for yes and no breakfast
    yes_lockdown = df.loc[(df['site'] == site) & (df['Lockdown'] == "Lockdown"), ["Lockdown", "NO2"]]
    no_lockdown = df.loc[(df['site'] == site) & (df['Lockdown'] == "No-lockdown"), ["Lockdown", "NO2"]]

    yes_lockdown.Lockdown = yes_lockdown.Lockdown.map({"No-lockdown": 0, "Lockdown":1})
    no_lockdown.Lockdown = no_lockdown.Lockdown.map({"No-lockdown": 0, "Lockdown":1})
    
    # Create ColumnDataSource objects
    source_yes = ColumnDataSource(yes_lockdown)
    source_no = ColumnDataSource(no_lockdown)

    p = figure(title = f"Relation between NO2 emission and lockdown, site {site} - {year}.",plot_width = 750, plot_height = 400, tools="pan, hover, zoom_in, zoom_out, yzoom_in, yzoom_out")


    # Create the points for patient who have died
    points = p.scatter(jitter('Lockdown', 0.1), 'NO2', source=source_yes, color = "green", marker = "dot", size = 30, legend_label = f"Lockdown: yes (mean: {round(yes_lockdown['NO2'].mean(),2)})", alpha = 0.7)
    # Create the points for patient who survived
    points2 = p.scatter(jitter('Lockdown', 0.1), 'NO2', source=source_no, color = "red", marker = "dot", size = 30, legend_label = f"Lockdown: no (mean: {round(no_lockdown['NO2'].mean(),2)})", alpha = 0.5)

    # Set labels
    p.xaxis.axis_label = 'Lockdown (0 = No-lockdown, 1 = lockdown))'
    # Use regex to grab the info about what was measured
    p.yaxis.axis_label = 'NO2 emission (µg/m³)'
    
    p.xaxis.ticker = [0, 1]

    # Make legend interactive
    p.legend.location = "top_center"
    p.legend.click_policy="hide"
    

    return p 

# p = create_scatter(tidy_2021, "NL10107")


# layout = pn.interact(create_scatter, site=autocomplete_station, year=select_year)
# pn.Row(pn.Column(layout[0], layout[1]))

pn.Column(pn.Row(autocomplete_station, select_year),
            pn.Row(create_scatter))

Column
    [0] Row
        [0] AutocompleteInput(name='Station', options=['NL10107', 'NL10131', ...], placeholder='ID of station..', value='NL10107')
        [1] Select(name='Year', options=['2020', '2021'], value='2020')
    [1] Row
        [0] ParamFunction(function)

The scatter plot displays the NO2 emission values for both no-lockdown and lockdown. The station and year of interest can be selected with the widgets. From the scatter plot we can conclude that for most stations the mean NO2 emission value is higher during the lockdown periods compared to no-lockdown periods. 

## <a id="/15">2.5 Distribution NO2 over the lockdown states - violin plot</a>  

In [127]:
# discrete_slider = pn.widgets.DiscreteSlider(name='Number of stations', options=[1, 2, 3, 4], value=3)
select_n_stations = pn.widgets.Select(name="Number of stations",
                                options=["All", "1", "2", "3", "4"])

years_df = {"2020":tidy_2020,
           "2021":tidy_2021}

select_year2 = pn.widgets.Select(name="Year",
                                options=["2020", "2021"])


def violin_plot(n_stations, year="2020"):
    """
    Create a violin plot.
    
    :parameters
    -----------
    n_stations - int
        Number of stations need to be displayed
    year - str
        Year
    """
    df = years_df[year]
    
    def get_plot(n_stations):
        if n_stations == "All":
            df_stations = df
            p = hv.Violin(df_stations, 
              kdims=['Lockdown'], vdims="NO2")
            p.opts(title=f"Violin plot for {n_stations.lower()} stations - {year}", 
                   width=750, height=600, ylabel = "NO2 (µg/m³)",
                  violin_color=dim('Lockdown'))
        else:
            n_stations = int(n_stations)
            stations = list(pd.unique(df.site)) 
            random_stations = random.sample(stations, n_stations)
            df_stations = df[df.site.isin(random_stations)]
            p = hv.Violin(df_stations, 
              kdims=['site', 'Lockdown'], vdims="NO2")
            p.opts(title=f"Violin plot for station {random_stations} - {year}", 
                   width=750, height=600, ylabel = "NO2 (µg/m³)",
                  violin_color=dim('Lockdown'))
        return p
        
    p = get_plot(n_stations)
    
    return p
    

layout = pn.interact(violin_plot, n_stations = select_n_stations, year=select_year2)
pn.Column(pn.Row(layout[0][0], layout[0][1]),
         pn.Row(layout[1]))

Column
    [0] Row
        [0] Select(name='Number of stations', options=['All', '1', '2', ...], value='All')
        [1] Select(name='Year', options=['2020', '2021'], value='2020')
    [1] Row
        [0] Row
            [0] HoloViews(Violin, name='interactive03578')

# <a id="/16">3 Check assumptions</a>  

To be able to test whether or not there is a difference in NO2 emission before and during a lockdown, a T-test or ANOVA could be used. However, both tests make a few assumptions, independence, normality, and homogeneity of variances. Before performing such a test tease assumptions should be test.

## <a id="/17">3.1 Check for normality - QQ plot</a>  

In [128]:
from scipy.stats import iqr # iqr is the Interquartile Range function
from scipy.stats import norm
def Q_Q_components(y, est = 'robust', **kwargs):
    """
    Calculate the componts of a QQ plot for a specified estimation method.
    
    :parameters
    -----------
    y - array like
        data array
    est - str
         Estimation method for normal parameters mu and sigma:
         either 'robust' (default), or 'ML' (Maximum Likelihood), or 'preset' (given values)
         If est='preset' than the optional parameters mu, sigma must be provided
         
    :returns
    --------
    z_th - array
        theoretical quantiles
    z_i - array
        sample quantiles
    CI_lower - array
        Confidence interval lower
    CI_upper - array
        Confidence interval upper
    """
    # First, get the optional arguments mu and sigma:
    mu_0 = kwargs.get('mu', None)
    sigma_0 = kwargs.get('sigma', None)
    
    n = len(y)
    
    # Calculate order statistic:
    y_os = np.sort(y)
  
    # Estimates of mu and sigma:
    # ML estimates:
    mu_ML = y.mean()
    sigma2_ML = np.mean((y.values - 6.66)**2, where=~np.isnan(y))
    
    sigma_ML = np.sqrt(sigma2_ML) # biased estimate
    s2 = n/(n-1) * sigma2_ML
    s = np.sqrt(s2) # unbiased estimate
    # Robust estimates:
    mu_R = y.median()
    sigma_R = iqr(y, nan_policy = 'omit')/1.349

    # Assign values of mu and sigma for z-transform:
    if est == 'ML':
        mu, sigma = mu_ML, s
    elif est == 'robust':
        mu, sigma = mu_R, sigma_R
    elif est == 'preset':
        mu, sigma = mu_0, sigma_0
    else:
        print('Wrong estimation method chosen!')
        
    # Perform z-transform: sample quantiles z.i
    z_i = (y_os - mu)/sigma

    # Calculate cumulative probabilities p.i:
    i = np.array(range(n)) + 1
    p_i = (i - 0.5)/n

    # Calculate theoretical quantiles z.(i):
    from scipy.stats import norm
    z_th = norm.ppf(p_i, 0, 1)

    # Calculate SE or theoretical quantiles:
    SE_z_th = (1/norm.pdf(z_th, 0, 1)) * np.sqrt((p_i * (1 - p_i)) / n)

    # Calculate 95% CI of diagonal line:
    CI_upper = z_th + 1.96 * SE_z_th
    CI_lower = z_th - 1.96 * SE_z_th

    return z_th, z_i, CI_lower, CI_upper

In [129]:
def Q_Q_plot(y, sample, est = "ML"):
    """
    Create a QQ plot that checks if a dataset has a normal distribution.
    
    :parameters
    -----------
    y - array like
        Data
    sample - str
        Name of the sample
    est - str
        Estimation method - optional
    """
    z_th, z_i, CI_lower, CI_upper = Q_Q_components(y, est = 'ML')

    p = figure(title = f"Q-Q plot: normality NO2 emission - {sample}",plot_width = 750, plot_height = 400, tools="pan, hover, zoom_in, zoom_out, yzoom_in, yzoom_out")


    # Create the points of the experimental daata
    points = p.scatter(z_th, z_i, color = "black", marker = "dot", size = 30, legend_label = "experimental data", alpha = 0.7)
    line_normal = p.line(z_th, z_th, color = "red", legend_label = "normal line", line_dash = "dashdot")
    # Create the lines for the 95% CI interval
    line_lower = p.line(z_th, CI_lower, color = "blue", legend_label = "95% CI", line_dash = "dashdot")
    line_upper = p.line(z_th, CI_upper, color = "blue", legend_label = "95% CI", line_dash = "dashdot")

    # Set labels
    p.xaxis.axis_label = 'Theoretical quantiles, Z(i)'
    p.yaxis.axis_label = 'Sample quantiles, Zi'


    # Make legend interactive
    p.legend.location = "top_left"
    p.legend.click_policy="hide"
    return p

In [130]:
# Get Q-Q plot data
y_2020_lockdown = tidy_2020.loc[(tidy_2020['Lockdown'] == "Lockdown"), "NO2"]

qq_2020_lockdown = Q_Q_plot(y_2020_lockdown, "2020, lockdown")

# show(qq_2020_lockdown)

![qq plot](imgs/QQ_2020_lockdown.png)

In [131]:
y_2020_no_lockdown = tidy_2020.loc[(tidy_2020['Lockdown'] == "No-lockdown"), "NO2"]

qq_2020_no_lockdown = Q_Q_plot(y_2020_no_lockdown, "2020, no-lockdown")
# show(qq_2020_no_lockdown)

![qq plot](imgs/QQ_2020_no_lockdown.png)

In [132]:
# Get Q-Q plot data
y_2021_lockdown = tidy_2021.loc[(tidy_2021['Lockdown'] == "Lockdown"), "NO2"]

qq_2021_lockdown = Q_Q_plot(y_2021_lockdown, "2021, lockdown")
# show(qq_2021_lockdown)

![qq plot](imgs/QQ_2021_lockdown.png)

In [133]:
y_2021_no_lockdown = tidy_2021.loc[(tidy_2021['Lockdown'] == "No-lockdown"), "NO2"]

qq_2021_no_lockdown = Q_Q_plot(y_2021_no_lockdown, "2021, no-lockdown")
# show(qq_2021_no_lockdown)

![qq plot](imgs/QQ_2021_no_lockdown.png)

A Q-Q plot is used to check for normality because there is a large sample size (>50). A Q-Q plot is made for both years and lockdown states. When inspecting the different Q-Q plots it can be concluded that none of the samples for both years is normally distributed, which means that a T-test or ANOVA can't be used to test for differences between the samples.

## <a id="/18">3.2 Check for homogeneity - Levene's Test of Equality</a>  

In [134]:
from scipy.stats import shapiro, levene, bartlett, f_oneway
_, p_val_2020 = levene(y_2020_lockdown, y_2020_no_lockdown, center='median')
_, p_val_2021 = levene(y_2021_lockdown, y_2021_no_lockdown, center='median')

def print_levene_test(p_val: float, sample:str, alpha:float) -> None:
    """
    Print the results of the level Levene's test of equality.
    
    :parameters
    -----------
    p_val - float
        P value of the test
    sample - str
        Name of the sample that was tested
    alpha - float
        Alpha value to determine significance
        
    """
    text = f"""\
    Levene’s Test of Equality (Homogeneity)::
    Sample: {sample}
    Null hypothesis: equal varaince amongst groups.
    P value: {p_val}.
    """
    if p_val > alpha:
        text += "p-value > 0.05, we accept the null hypothesis which means that the variance between groups is not significantly different (equal variance).\n"
    elif p_val <= alpha:
        text += "p-value < 0.05, we reject the null hypothesis which means that the variance between groups is significantly different (no equal variance).\n"
    print(text)
    
print_levene_test(p_val_2020, "2020", 0.05)
print_levene_test(p_val_2021, "2021", 0.05)

    Levene’s Test of Equality (Homogeneity)::
    Sample: 2020
    Null hypothesis: equal varaince amongst groups.
    P value: 0.0.
    p-value < 0.05, we reject the null hypothesis which means that the variance between groups is significantly different (no equal variance).

    Levene’s Test of Equality (Homogeneity)::
    Sample: 2021
    Null hypothesis: equal varaince amongst groups.
    P value: 0.0.
    p-value < 0.05, we reject the null hypothesis which means that the variance between groups is significantly different (no equal variance).



## <a id="/19">3.3 Summary</a>  

The Levene's test of equality was used to check if the different groups contained equal variance. Median was used as center, because it is recommended for non-normal data[1]. Equal variance of NO2 emission levels was tested for the two lockdown states in both years. For both instances the null hypothesis was rejected and it was concluded that the groups did not have equal variance. 

In summary, the two different groups are not normally distributed and do not have equal variance. This means that a T-test or ANOVA can't be used to test if there is a significant difference between the two groups. However, a non-parametric test can be used.  

For example the Mann-Whitney U test. "The Mann-Whitney U Test is a nonparametric version of the independent samples t-test. The test primarily deals with two independent samples that contain ordinal data"[2].

[1] https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.levene.html  
[2] https://corporatefinanceinstitute.com/resources/knowledge/other/nonparametric-tests/

# <a id="/20">4 Test for differences</a>  

## Combine the two data sets.

In [135]:
NO2_lockdown_data = pd.concat([tidy_2020, tidy_2021], ignore_index=True, axis=0)
NO2_lockdown_data.head(20)

,date_start,date_end,Lockdown,covid_patients,site,NO2
0,2020-01-01 00:00:00,2020-01-01 01:00:00,No-lockdown,0.0,NL10107,38.78
1,2020-01-01 01:00:00,2020-01-01 02:00:00,No-lockdown,0.0,NL10107,38.31
2,2020-01-01 02:00:00,2020-01-01 03:00:00,No-lockdown,0.0,NL10107,37.06
3,2020-01-01 03:00:00,2020-01-01 04:00:00,No-lockdown,0.0,NL10107,35.16
4,2020-01-01 04:00:00,2020-01-01 05:00:00,No-lockdown,0.0,NL10107,32.31
5,2020-01-01 05:00:00,2020-01-01 06:00:00,No-lockdown,0.0,NL10107,29.85
6,2020-01-01 06:00:00,2020-01-01 07:00:00,No-lockdown,0.0,NL10107,28.38
7,2020-01-01 07:00:00,2020-01-01 08:00:00,No-lockdown,0.0,NL10107,30.11
8,2020-01-01 08:00:00,2020-01-01 09:00:00,No-lockdown,0.0,NL10107,27.51
9,2020-01-01 09:00:00,2020-01-01 10:00:00,No-lockdown,0.0,NL10107,22.61


In [136]:
# The two samples are not of equal size.
NO2_lockdown_data.Lockdown.value_counts()

No-lockdown    520128
Lockdown       202272
Name: Lockdown, dtype: int64

## Create data frame qith equal sample size

In [137]:
def create_equal_sample_df(df: pd.DataFrame, by: str) -> pd.DataFrame:
    """
    Create a data frame with equal samples sizes for each group in a column by removing random rows.
    The number of rows removed is deterimend by substracting the minimum value count from each value count.
    
    :parameters
    -----------
    df - pandas.DataFrame
        A pandas data frame
    by - String
        Column in the DataFrame used as samples
        
    :returns
    --------
    eq_samples - pandas.DataFrame
        Data frame with equal sample sizes.
    """
    np.random.seed(10)
    # Check how many samples each group has
    sample_sizes = df[by].value_counts()
    
    # Get min sample value:
    min_n = sample_sizes.min()

    n_remove = {}
    # Calculate how many rows should be removed per group.
    for idx, row in sample_sizes[sample_sizes.index != sample_sizes.idxmin()].iteritems():
        n_remove[idx] = row - min_n

    eq_samples = df.copy()

    # Drop random rows for both groups
    for key, value in n_remove.items():
        eq_samples.drop(eq_samples[eq_samples[by] == key].sample(n=value).index, inplace = True)

    return eq_samples
    

eq_samples = create_equal_sample_df(NO2_lockdown_data, 'Lockdown')
eq_samples.Lockdown.value_counts()

Lockdown       202272
No-lockdown    202272
Name: Lockdown, dtype: int64

In [138]:
from scipy.stats import mannwhitneyu

yes_lockdown = eq_samples.loc[eq_samples.Lockdown == "Lockdown", "NO2"].values
no_lockdown = eq_samples.loc[eq_samples.Lockdown == "No-lockdown", "NO2"].values

_, p_val = mannwhitneyu(yes_lockdown, no_lockdown, alternative="two-sided")

In [139]:
def print_manwhitneyu_test(p_val: float, alpha:float) -> None:
    """
    Print the results of the level Levene's test of equality.
    
    :parameters
    -----------
    p_val - float
        P value of the test
    alpha - float
        Alpha value to determine significance
        
    """
    text = f"""\
    Mann-Whitney U test:
    Null hypothesis: the sum of the rankings are equal in the two groups.
    P value: {p_val}.
    """
    if p_val > alpha:
        text += "p-value > 0.05, we accept the null hypothesis which means that the sum of rankings is NOT significantly different.\n"
    elif p_val <= alpha:
        text += "p-value < 0.05, we reject the null hypothesis which means that the sum of rankings between groups IS significantly different.\n"
    print(text)
    
print_manwhitneyu_test(p_val, 0.05)

    Mann-Whitney U test:
    Null hypothesis: the sum of the rankings are equal in the two groups.
    P value: 0.0.
    p-value < 0.05, we reject the null hypothesis which means that the sum of rankings between groups IS significantly different.



# <a id="/21">5 Conclusion</a>  

The p-value resulting from the Mann-Whitney U test is 0.0, which is belowe the alpha value of 0.05. This means that we can reject the null hypothesis and accept the alternative hypothesis which states that the sum of rankings between group is significanlty different. 

Meaning that the lcodkowns in the Netherlands causes the NO2 emission to change. 

# <a id="/22">6 Dashboard code</a>  

## Combine data with meta data

In [140]:
combined_df = NO2_lockdown_data.merge(df_meta_data, how = "inner", left_on = ["site"], right_index = True)
combined_df.head(10)

,date_start,date_end,Lockdown,covid_patients,site,NO2,Stationsnaam,Latitude,Longitude,Stationsgebied,Stationstype,Meetprincipe,Meetopstelling
0,2020-01-01 00:00:00,2020-01-01 01:00:00,No-lockdown,0.0,NL10107,38.78,Posterholt-Vlodropperweg,51.119194,6.042399,regionaal,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
1,2020-01-01 01:00:00,2020-01-01 02:00:00,No-lockdown,0.0,NL10107,38.31,Posterholt-Vlodropperweg,51.119194,6.042399,regionaal,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
2,2020-01-01 02:00:00,2020-01-01 03:00:00,No-lockdown,0.0,NL10107,37.06,Posterholt-Vlodropperweg,51.119194,6.042399,regionaal,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
3,2020-01-01 03:00:00,2020-01-01 04:00:00,No-lockdown,0.0,NL10107,35.16,Posterholt-Vlodropperweg,51.119194,6.042399,regionaal,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
4,2020-01-01 04:00:00,2020-01-01 05:00:00,No-lockdown,0.0,NL10107,32.31,Posterholt-Vlodropperweg,51.119194,6.042399,regionaal,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
5,2020-01-01 05:00:00,2020-01-01 06:00:00,No-lockdown,0.0,NL10107,29.85,Posterholt-Vlodropperweg,51.119194,6.042399,regionaal,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
6,2020-01-01 06:00:00,2020-01-01 07:00:00,No-lockdown,0.0,NL10107,28.38,Posterholt-Vlodropperweg,51.119194,6.042399,regionaal,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
7,2020-01-01 07:00:00,2020-01-01 08:00:00,No-lockdown,0.0,NL10107,30.11,Posterholt-Vlodropperweg,51.119194,6.042399,regionaal,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
8,2020-01-01 08:00:00,2020-01-01 09:00:00,No-lockdown,0.0,NL10107,27.51,Posterholt-Vlodropperweg,51.119194,6.042399,regionaal,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser
9,2020-01-01 09:00:00,2020-01-01 10:00:00,No-lockdown,0.0,NL10107,22.61,Posterholt-Vlodropperweg,51.119194,6.042399,regionaal,achtergrond,Chemiluminescentie,Teledyne API 200E chemiluminescent Nox Analyser


## <a id="/23">6.1 geojson data</a> 

In [141]:
# Link: https://www.webuildinternet.com/2015/07/09/geojson-data-of-the-netherlands/

class GeoDataParser:
    """
    Parse geojson data. Seperate several areas from one file.

    ...

    Attributes
    ----------
    geojson : str
        Location of geosjon file
    outdir: str
        Location of the output directory


    Methods
    -------
    make_data_dir(self, path) -> None:
        Create a directory in the system to store the data, if it doesn't exist yet.
    read_geojson(self) -> file:
        Read in a geosjon file
    extract_features(self, geo_data) -> None:
        Extract the different features from a geosjon file.
    write_json_data(self, file, data) -> None:
        Write out json formatted data.
    """
    
    def __init__(self, geojson, outdir):
        self.geojson = Path(geojson)
        self.outdir = Path(outdir)
        
        self.make_data_dir(self.outdir)
        
    def make_data_dir(self, path: Path) -> None:
        """
        Create a directory (if it does not exisit yet) to store the 
        data.
        
        :parameter
        ----------
        path - Path
            directory path

        :Excepts
        --------
        FileExistsError
            The directory already exists
        """
        try:
            path.mkdir(parents=True, exist_ok=False)
        except FileExistsError:
            print(f"[{self.make_data_dir.__name__}] Folder: {path} already exists.")
            
    
    def read_geojson(self) -> dict:
        """
        Read in geojson data.
        
        :returns
        --------
        json_data - dict
            Geojson data in dictionary form
        """
        file = open(self.geojson)
        geo_data = json.load(file)
        file.close()
        return geo_data
    
    def extract_features(self, geo_data: dict) -> None:
        """
        Extract the different features from a geosjon file.
        
        :parameters
        -----------
        geo_data - dict
            GeoJson data inside a dictionary
        """
        for i in range(len(geo_data['features'])):
            data = {"type": 'FeatureCollection'}
            data['features'] = [geo_data['features'][i]]

            # Extract name of province
            name = geo_data['features'][i]['properties']['name'].lower()
            file = self.outdir / f"{name}.geojson"
            
            self.write_json_data(file, data)
            
        print("Done extracting features..")


    def write_json_data(self, file: Path, data: dict) -> None:
        """
        Write out json formatted data.
        
        :parameters
        -----------
        file - Path
            File path
        data - dict
            geosjon data
        """
        with open(file, "w") as outfile:
            json.dump(data, outfile)
            
    
geojson_file = config["geojson"]
        
geo_parser = GeoDataParser(geojson_file, "geojson")

data = geo_parser.read_geojson()

geo_parser.extract_features(data)

[make_data_dir] Folder: geojson already exists.
Done extracting features..


## <a id="/24">6.2 NO2 emission map</a>  

In [142]:
class NO2Map:
    """
    Parent class for geo maps based upon NO2 emission.

    ...


    Methods
    -------
    get_map(self, lat, long, *, zoom_start, tiles) -> folium.Map:
        Create a folium map
    make_legend(labels, colors) -> MacroElement:
        Create a legend object
    """
    
    def get_map(self, lat, long, *, zoom_start = 7.5, tiles = "CartoDB Positron") -> folium.Map:
        """
        Create a folium map.
        
        :parameters
        -----------
        lat - float
            Latitude
        long - float
            Longitude
        zoom_start - float
            Starting zoom level - optional
        tiles - str
            Kind of map - optional
            
        :returns
        --------
        fm_map - folium.Map
            A folium map
        """
        # Create a map and start at the mean Lattitude and longitude values
        fm_map = folium.Map(location=[lat, long], zoom_start=zoom_start,
                            tiles=tiles)
        return fm_map
    
    def make_legend(self, labels, colors) -> MacroElement:
        """
        Create a legend for a folium map vased on input labels and colors.
        
        :parameters
        ----------
        labels - list
            Legend labels
        colors - list
            Legend colors
            
        :returns
        --------
        legend - MacroElement
            Legend in html form
        """
        if len(labels) != len(colors):
            raise ValueError('labels and colors must have the same length')

        legend_content = ""
        for l_color, l_text in zip(colors, labels):
            legend_content += f"<li><span style='background:{l_color};opacity:0.7;'></span>{l_text}</li>\n"

        template = """
        {% macro html(this, kwargs) %}

        <!doctype html>
        <html lang="en">
        <head>
          <meta charset="utf-8">
          <meta name="viewport" content="width=device-width, initial-scale=1">
          <title>jQuery UI Draggable - Default functionality</title>
          <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

          <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
          <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>

          <script>
          $( function() {
            $( "#maplegend" ).draggable({
                            start: function (event, ui) {
                                $(this).css({
                                    right: "auto",
                                    top: "auto",
                                    bottom: "auto"
                                });
                            }
                        });
        });

          </script>
        </head>
        <body>


        <div id='maplegend' class='maplegend' 
            style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
             border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>

        <div class='legend-title'>Legend</div>
        <div class='legend-scale'>
          <ul class='legend-labels'>
        """ + legend_content + """
          </ul>
        </div>
        </div>

        </body>
        </html>

        <style type='text/css'>
          .maplegend .legend-title {
            text-align: left;
            margin-bottom: 5px;
            font-weight: bold;
            font-size: 90%;
            }
          .maplegend .legend-scale ul {
            margin: 0;
            margin-bottom: 5px;
            padding: 0;
            float: left;
            list-style: none;
            }
          .maplegend .legend-scale ul li {
            font-size: 80%;
            list-style: none;
            margin-left: 0;
            line-height: 18px;
            margin-bottom: 2px;
            }
          .maplegend ul.legend-labels li span {
            display: block;
            float: left;
            height: 16px;
            width: 30px;
            margin-right: 5px;
            margin-left: 0;
            border: 1px solid #999;
            }
          .maplegend .legend-source {
            font-size: 80%;
            color: #777;
            clear: both;
            }
          .maplegend a {
            color: #777;
            }
        </style>
        {% endmacro %}"""


        legend = MacroElement()
        legend._template = Template(template)

        return legend
    
    @staticmethod
    def add_legend(fm_map, legend) -> None:
        """
        Add a legend to a folium map.
        
        :parameters
        -----------
        fm_map - folium.Map
            Folium map
        legend - MacroElement
            A legend describing the map.
        """
        fm_map.get_root().add_child(legend)
        
        
legend_text = ["Good (0-40 µg/m³)", "Moderate (40-75 µg/m³)", "Not good (75-125 µg/m³)", "Bad (125-200 µg/m³)", "Very bad(>200 µg/m³)"]
legend_colors = ["blue", "yellow", "orange", "red", "purple"]

base_NO2_map = NO2Map()
base_map = base_NO2_map.get_map(df_meta_data.Latitude.mean(), df_meta_data.Longitude.mean())
legend = base_NO2_map.make_legend(legend_text, legend_colors)
base_NO2_map.add_legend(base_map, legend)
base_map